In [1]:
!pip install -q google-generativeai pdfplumber python-docx plotly requests pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 66.3 MB/s eta 0:00:00


In [2]:
import google.generativeai as genai
from google.colab import userdata

# Access the API key from Colab Secrets Manager
GEMINI_KEY = userdata.get('GEMINI_API_KEY')

# Access RapidAPI Key from Colab Secrets Manager
RAPID_KEY = userdata.get('RAPID_API_KEY')


genai.configure(api_key=GEMINI_KEY)

Enter your RapidAPI Key:4720b91364msha9741c499196355p10b440jsned70d83a0c7f··········


In [3]:
import pdfplumber, docx
from google.colab import files

def extract_text(file_path):
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            return "\n".join(p.extract_text() or "" for p in pdf.pages)
    elif file_path.endswith(".docx"):
        doc = docx.Document(file_path)
        return "\n".join(p.text for p in doc.paragraphs)
    else:
        return open(file_path, "r", encoding="utf-8").read()

print("📂 Upload your resume file (.pdf or .docx)")
uploaded = files.upload()
fname = list(uploaded.keys())[0]
resume_text = extract_text(fname)
print("\n✅ Resume extracted! Showing first 500 chars:\n")
print(resume_text[:500])


📂 Upload your resume file (.pdf or .docx)


Saving mithil resume (4).pdf to mithil resume (4).pdf

✅ Resume extracted! Showing first 500 chars:

Mithil Bhosale
Thane, Maharashtra | +91-8928194137 | bhosalemithil30@gmail.com | LinkedIn
Final-year Computer Science Engineering (AIML) student with a strong foundation in Java, Python, and C, along
with hands-on experience in programming fundamentals, problem-solving, and developing technology-driven
solutions. Passionate about innovation, AI/ML, and building applications that create real-world impact while
continuously learning and growing as a developer.
EDUCATION
● B.E. IN CSE(Artificial In


In [4]:
from IPython.display import JSON
model = genai.GenerativeModel("gemini-2.0-flash")

user_name = input("👤 Name: ")
job_field = input("💼 Interested Field (e.g., Data Science, Marketing, Design): ")
skillset = input("🧩 List your known skills (comma-separated): ")

prompt = f"""
You are an AI career coach.
Analyze this candidate's résumé and provide a structured JSON output.

Candidate Info:
Name: {user_name}
Field of Interest: {job_field}
Self-Declared Skills: {skillset}

Resume Text:
{resume_text}

Return a JSON with:
{{
  "overall_skill_index": 0-100,
  "category_scores": {{
    "Frontend": number,
    "Backend": number,
    "AI/ML": number,
    "Design": number,
    "DevOps": number
  }},
  "strengths": [list of strings],
  "weaknesses": [list of strings],
  "summary": "1-sentence summary of profile"
}}
"""

response = model.generate_content(prompt)
print("✅ Gemini Response:\n")
print(response.text)

👤 Name: Mithil
💼 Interested Field (e.g., Data Science, Marketing, Design): Data Science
🧩 List your known skills (comma-separated): Java, Python
✅ Gemini Response:

```json
{
  "overall_skill_index": 75,
  "category_scores": {
    "Frontend": 70,
    "Backend": 75,
    "AI/ML": 85,
    "Design": 60,
    "DevOps": 70
  },
  "strengths": [
    "Strong AI/ML project experience, including disease detection and career coaching applications.",
    "Experience with a variety of ML models and frameworks (CNNs, LightGBM, TensorFlow, PyTorch).",
    "Full-stack capabilities demonstrated through project work and internship experience.",
    "Experience in both application development (Flutter, mobile) and web technologies (React, Node.js).",
    "Extracurricular activities show leadership and communication skills."
  ],
  "weaknesses": [
    "Limited information about specific contributions and impact within team projects.",
    "GPA could be improved to strengthen academic profile.",
    "The re

In [5]:
import json, plotly.graph_objects as go

# Extract the JSON string from the response text
json_string = response.text.strip()
if json_string.startswith("```json"):
    json_string = json_string[7:]
if json_string.endswith("```"):
    json_string = json_string[:-3]

data = json.loads(json_string)

labels = list(data["category_scores"].keys())
values = list(data["category_scores"].values())

fig = go.Figure()
fig.add_trace(go.Scatterpolar(
    r = values + [values[0]],
    theta = labels + [labels[0]],
    fill = 'toself',
    name = 'Skill Scores'
))
fig.update_layout(
    title = f"Skill Index: {data['overall_skill_index']}",
    polar=dict(radialaxis=dict(visible=True, range=[0,100])),
    showlegend=False,
    template="plotly_dark"
)
fig.show()

In [6]:
import requests

def fetch_jobs(role, location="India", limit=5):
    url = "https://jsearch.p.rapidapi.com/search"
    query = {"query": role, "num_pages": 1}
    headers = {
        "X-RapidAPI-Key": RAPID_KEY,
        "X-RapidAPI-Host": "jsearch.p.rapidapi.com"
    }
    r = requests.get(url, headers=headers, params=query)
    data = r.json()
    jobs = []
    for j in data.get("data", [])[:limit]:
        jobs.append({
            "title": j.get("job_title"),
            "company": j.get("employer_name"),
            "location": j.get("job_city"),
            "description": j.get("job_description", "")[:300] + "...",
            "url": j.get("job_apply_link")
        })
    return jobs

search_role = input("🔍 Enter job role to search (e.g., Data Analyst, ML Engineer): ")
jobs = fetch_jobs(search_role, limit=5)
print("✅ Jobs fetched:")
for j in jobs:
    print(f"- {j['title']} at {j['company']} ({j['location']})")


🔍 Enter job role to search (e.g., Data Analyst, ML Engineer): ML engineer
✅ Jobs fetched:
- ML Ops Engineer at Sev1Tech LLC (Arlington)
- Sr. Lead Machine Learning Engineer, Shopping (Remote-Eligible) at Capital One (McLean)
- AI/ML Engineer Level 2 at CACI (Suitland-Silver Hill)
- Generative AI & Machine Learning Engineer at Thomson Reuters (McLean)
- AI and ML Engineer Jobs at Booz Allen Hamilton (Alexandria)


In [7]:
prompt_jobs = f"""
You are an AI job matcher. Based on this user's résumé and skills, rate how well each job matches.

Candidate Résumé:
{resume_text}

Jobs:
{jobs}

Return JSON:
[
  {{
    "title": "Job Title",
    "company": "Company Name",
    "match_score": 0-100,
    "reason": "why this fits",
    "apply_link": "URL"
  }}
]
"""

response_jobs = model.generate_content(prompt_jobs)
print("✅ Gemini Job Match Results:\n")
print(response_jobs.text)


✅ Gemini Job Match Results:

```json
[
  {
    "title": "ML Ops Engineer",
    "company": "Sev1Tech LLC",
    "match_score": 75,
    "reason": "Mithil's experience with Python, Docker (from the Legacy Modernization project), and a strong foundation in ML models align well with the need to deploy, monitor, and optimize AI models. The focus on automating model deployment is also a good fit given his project work. While he is a student, his project experience demonstrates practical application of relevant skills. His skills in cloud technologies are demonstrated via certificates.",
    "apply_link": "https://www.linkedin.com/jobs/view/ml-ops-engineer-at-sev1tech-llc-4318935073?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic"
  },
  {
    "title": "Sr. Lead Machine Learning Engineer, Shopping (Remote-Eligible)",
    "company": "Capital One",
    "match_score": 60,
    "reason": "While Mithil possesses relevant ML skills (Python, Scikit-learn, TensorFlow, Kera

In [8]:
import pandas as pd

json_string = response_jobs.text.strip()
if json_string.startswith("```json"):
    json_string = json_string[7:]
if json_string.endswith("```"):
    json_string = json_string[:-3]

matches = json.loads(json_string)
df = pd.DataFrame(matches)
df = df.sort_values("match_score", ascending=False)
df.style.background_gradient(subset=["match_score"], cmap="Blues")

,title,company,match_score,reason,apply_link
3,Generative AI & Machine Learning Engineer,Thomson Reuters,78,"Mithil's work on SkillPilotAI, particularly the LLM-driven chatbot, directly aligns with Generative AI. His overall ML skillset and project experience make him a strong candidate. The description focuses on data quality improvement, which aligns with his ability to convert CSV/flat files into structured PostgreSQL schemas (from the Legacy Modernization project).",https://careers.thomsonreuters.com/us/en/job/JREQ193871/Generative-AI-Machine-Learning-Engineer?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic
0,ML Ops Engineer,Sev1Tech LLC,75,"Mithil's experience with Python, Docker (from the Legacy Modernization project), and a strong foundation in ML models align well with the need to deploy, monitor, and optimize AI models. The focus on automating model deployment is also a good fit given his project work. While he is a student, his project experience demonstrates practical application of relevant skills. His skills in cloud technologies are demonstrated via certificates.",https://www.linkedin.com/jobs/view/ml-ops-engineer-at-sev1tech-llc-4318935073?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic
2,AI/ML Engineer Level 2,CACI,70,"Mithil's AI/ML skills (TensorFlow, Keras, PyTorch, CNNs, RNNs, LSTM, LGBM classifier) and project experience (DigiFarmer, SkillPilotAI, Legacy Modernization) are relevant to the role. The Level 2 designation suggests some experience is required, but his internship and projects demonstrate a proactive approach to gaining practical skills. The biggest barrier is the TS/SCI security clearance requirement which most likely he doesn't possess, but he might be able to obtain it.",https://careers.caci.com/global/en/job/CACIGLOBAL319062EXTERNALENGLOBAL/AI-ML-Engineer-Level-2?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic
4,AI and ML Engineer Jobs,Booz Allen Hamilton,70,"Mithil's ability to conduct statistical analyses on workflow processes using ML techniques is highlighted in the job description. His project experiences, including DigiFarmer and SkillPilotAI, demonstrate practical application of ML for understanding and processing datasets. His programming skills (Python) and ML framework experience make him a suitable candidate. However, this job also requires a security clearance which might be a hurdle.",https://www.clearancejobs.com/jobs/8577608/ai-and-ml-engineer?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic
1,"Sr. Lead Machine Learning Engineer, Shopping (Remote-Eligible)",Capital One,60,"While Mithil possesses relevant ML skills (Python, Scikit-learn, TensorFlow, Keras, PyTorch, CNNs, RNNs, LSTM, LGBM classifier), the 'Sr. Lead' designation suggests a level of experience he currently lacks. However, his project work (DigiFarmer, SkillPilotAI) demonstrates practical application of ML to solve complex problems and build platforms, which is a good foundation. He may be a good fit once he gains a few years of experience. His experience in web technologies might also be valuable.",https://www.capitalonecareers.com/job/mclean/sr-lead-machine-learning-engineer-shopping-remote-eligible/1732/87800123136?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic


In [9]:
import datetime, json
fname = f"SkillPilotAI_{user_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}.json"
with open(fname, "w") as f:
    json.dump({"resume_analysis": data, "job_matches": matches}, f, indent=2)
print("✅ Saved:", fname)
from google.colab import files
files.download(fname)

✅ Saved: SkillPilotAI_Mithil_20251030_1653.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import getpass, requests, google.generativeai as genai
from google.colab import userdata

COURSERA_KEY = userdata.get('COURSERA_API_KEY')

🔑 Enter your Coursera RapidAPI key: ··········


In [16]:
def fetch_coursera_courses(query="data science", limit=8):
    url = "https://collection-for-coursera-courses.p.rapidapi.com/rapidapi/course/get_institution.php"
    headers = {
        "x-rapidapi-host": "collection-for-coursera-courses.p.rapidapi.com",
        "x-rapidapi-key": COURSERA_KEY
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Coursera courses: {e}")
        return []
    except json.JSONDecodeError:
        print("Error decoding JSON from Coursera API. The data might not be in the expected format.")
        print("Response content:", response.text[:500]) # Print first 500 characters for debugging
        return []


    courses = []
    # The API response structure might vary, inspect the actual response to adjust this loop
    # Assuming 'data' is a list of course dictionaries based on the original code attempt
    for c in data.get("data", [])[:limit]:
        title = c.get("title") or c.get("name", "Unknown")
        link = c.get("url", "N/A")
        university = c.get("university") or c.get("institution") or "Coursera"
        courses.append({
            "title": title,
            "university": university,
            "url": link,
            "platform": "Coursera"
        })

    return courses

In [12]:
model = genai.GenerativeModel("gemini-2.0-flash")

weaknesses = ", ".join(data["weaknesses"])
interest = job_field

# Fetch real courses
real_courses = fetch_coursera_courses(interest, limit=8)

prompt_courses = f"""
You are an AI career mentor that recommends Coursera courses based on user résumé insights.

User Weaknesses: {weaknesses}
User Strengths: {data['strengths']}
Field of Interest: {interest}
Skill Index: {data['overall_skill_index']}

Here are some real Coursera courses:
{real_courses}

From this list, choose and return up to 5 that best help the user upskill.
Output pure JSON like:
[
  {{
    "title": "Course Name",
    "reason": "Why this is ideal for user",
    "university": "Offering University",
    "url": "Course URL",
    "platform": "Coursera"
  }}
]
"""

response_courses = model.generate_content(prompt_courses)
print("✅ Gemini Course Recommendations (Filtered):\n")
print(response_courses.text)


✅ Gemini Course Recommendations (Filtered):

Okay, I need a list of Coursera courses to recommend *from*.  You provided an empty list: `[]`.  I *cannot* select from an empty list.

**Please provide the list of Coursera courses you'd like me to choose from.**  Once you give me that list, I will analyze it in relation to the user's strengths, weaknesses, field of interest (Data Science), and skill index, and return a JSON array with up to 5 course recommendations.



In [15]:
import json, pandas as pd

json_string = response_courses.text.strip()
print("Raw response text:", json_string) # Keep this for debugging if needed

# Find the start and end indices of the JSON block
json_start = json_string.find("```json")
json_end = json_string.rfind("```")

if json_start != -1 and json_end != -1:
    # Extract the JSON string between the markers
    json_string = json_string[json_start + 7:json_end].strip()
    try:
        courses = json.loads(json_string)
        df_courses = pd.DataFrame(courses)
        df_courses.style.set_properties(**{'text-align': 'left'}).background_gradient(subset=[], cmap="Greens")
    except json.JSONDecodeError:
        print("Error decoding JSON from the extracted string.")
        print("Extracted JSON string:", json_string)
else:
    # If markers are not found, it might not be a JSON response
    print("JSON markers not found in the response. The response might not be in the expected JSON format.")
    # Optionally, you could try to parse the whole string if you expect pure JSON sometimes
    # try:
    #     courses = json.loads(json_string)
    #     df_courses = pd.DataFrame(courses)
    #     df_courses.style.set_properties(**{'text-align': 'left'}).background_gradient(subset=[], cmap="Greens")
    # except json.JSONDecodeError:
    #     print("Error decoding JSON from the entire response string.")
    #     print("Full response text:", json_string)

Raw response text: Okay, I need a list of Coursera courses to recommend *from*.  You provided an empty list: `[]`.  I *cannot* select from an empty list.

**Please provide the list of Coursera courses you'd like me to choose from.**  Once you give me that list, I will analyze it in relation to the user's strengths, weaknesses, field of interest (Data Science), and skill index, and return a JSON array with up to 5 course recommendations.
JSON markers not found in the response. The response might not be in the expected JSON format.


In [17]:
import getpass, requests, google.generativeai as genai
from google.colab import userdata

YOUTUBE_KEY = userdata.get('YOUTUBE_API_KEY')

🔑 Enter your youtube138 RapidAPI key: ··········


In [18]:
def fetch_youtube_videos(query, limit=5):
    url = "https://youtube138.p.rapidapi.com/auto-complete/"
    headers = {
        "x-rapidapi-host": "youtube138.p.rapidapi.com",
        "x-rapidapi-key": YOUTUBE_KEY
    }
    params = {"q": query, "hl": "en", "gl": "US"}

    r = requests.get(url, headers=headers, params=params)
    data = r.json()

    suggestions = []
    for item in data.get("results", [])[:limit]:
        title = item.get("query")
        if title:
            suggestions.append({
                "title": title,
                "url": f"https://www.youtube.com/results?search_query={title.replace(' ', '+')}"
            })
    return suggestions

In [19]:
user_skills = input("🧠 Enter your skills (comma-separated): ")
user_experience = input("💼 Enter your experience level (e.g., Fresher, 2 years, Intermediate): ")
career_goal = input("🎯 Enter your career goal (e.g., Data Scientist, UI/UX Designer, Cloud Engineer): ")

🧠 Enter your skills (comma-separated): python, java, php
💼 Enter your experience level (e.g., Fresher, 2 years, Intermediate): fresher
🎯 Enter your career goal (e.g., Data Scientist, UI/UX Designer, Cloud Engineer): cloud engineer


In [20]:
youtube_resources = fetch_youtube_videos(career_goal, limit=5)
print("✅ YouTube Suggestions:")
for y in youtube_resources:
    print("-", y["title"], "→", y["url"])

✅ YouTube Suggestions:


In [21]:
model = genai.GenerativeModel("gemini-2.0-flash")

prompt_path = f"""
You are an AI career coach helping users achieve their dream role.

User Info:
Skills: {user_skills}
Experience: {user_experience}
Career Goal: {career_goal}

Based on this, create a 3-stage career roadmap:
1. Foundation stage — core concepts to master.
2. Intermediate stage — projects, certifications, and practice.
3. Advanced stage — specialization and job preparation.

In each stage, recommend specific skills to learn and real YouTube learning links from this data:
{youtube_resources}

Return output as structured JSON:
{{
  "career_goal": "{career_goal}",
  "roadmap": [
    {{
      "stage": "Foundation",
      "skills_to_learn": [list],
      "youtube_resources": [{{"title": "Video Title", "url": "Video URL"}}],
      "estimated_duration": "e.g. 2 months"
    }},
    {{
      "stage": "Intermediate",
      "skills_to_learn": [list],
      "youtube_resources": [{{"title": "Video Title", "url": "Video URL"}}],
      "estimated_duration": "e.g. 3 months"
    }},
    {{
      "stage": "Advanced",
      "skills_to_learn": [list],
      "youtube_resources": [{{"title": "Video Title", "url": "Video URL"}}],
      "estimated_duration": "e.g. 4 months"
    }}
  ]
}}
"""

response_path = model.generate_content(prompt_path)
print("✅ Gemini Career Roadmap:\n")
print(response_path.text)


✅ Gemini Career Roadmap:

```json
{
  "career_goal": "cloud engineer",
  "roadmap": [
    {
      "stage": "Foundation",
      "skills_to_learn": [
        "Linux Fundamentals (Command Line, File System)",
        "Networking Basics (TCP/IP, DNS, Routing, Subnetting)",
        "Cloud Computing Concepts (IaaS, PaaS, SaaS, Public, Private, Hybrid)",
        "Basic Security Principles (Authentication, Authorization, Encryption)",
        "Version Control with Git",
        "Introduction to One Cloud Platform (AWS, Azure, or GCP)"
      ],
      "youtube_resources": [],
      "estimated_duration": "2-3 months"
    },
    {
      "stage": "Intermediate",
      "skills_to_learn": [
        "Deep Dive into Chosen Cloud Platform (AWS, Azure, or GCP) Services",
        "Infrastructure as Code (IaC) with Terraform or CloudFormation",
        "Containerization with Docker",
        "Orchestration with Kubernetes",
        "Basic Scripting (Bash, Python) for Automation",
        "Cloud Security Be

In [22]:
import json, pandas as pd

json_string = response_path.text.strip()

# Find the start and end indices of the JSON block
json_start = json_string.find("```json")
json_end = json_string.rfind("```")

if json_start != -1 and json_end != -1:
    # Extract the JSON string between the markers
    json_string = json_string[json_start + 7:json_end].strip()
else:
    # If markers are not found, try to parse the whole string (might be pure JSON)
    pass # Or handle this case as an error if expected to always have markers

career_data = json.loads(json_string)
df_path = pd.DataFrame(career_data["roadmap"])
df_path.style.set_properties(**{'text-align': 'left'})

,stage,skills_to_learn,youtube_resources,estimated_duration
0,Foundation,"['Linux Fundamentals (Command Line, File System)', 'Networking Basics (TCP/IP, DNS, Routing, Subnetting)', 'Cloud Computing Concepts (IaaS, PaaS, SaaS, Public, Private, Hybrid)', 'Basic Security Principles (Authentication, Authorization, Encryption)', 'Version Control with Git', 'Introduction to One Cloud Platform (AWS, Azure, or GCP)']",[],2-3 months
1,Intermediate,"['Deep Dive into Chosen Cloud Platform (AWS, Azure, or GCP) Services', 'Infrastructure as Code (IaC) with Terraform or CloudFormation', 'Containerization with Docker', 'Orchestration with Kubernetes', 'Basic Scripting (Bash, Python) for Automation', 'Cloud Security Best Practices']",[],3-4 months
2,Advanced,"['Specialization in a Cloud Domain (e.g., DevOps, Data Engineering, Security)', 'Advanced Cloud Networking (VPC configuration, Load Balancing)', 'Monitoring and Logging (CloudWatch, Azure Monitor, GCP Stackdriver)', 'CI/CD Pipelines (Jenkins, GitLab CI, AWS CodePipeline)', 'Serverless Computing (AWS Lambda, Azure Functions, GCP Cloud Functions)', 'Prepare for Cloud Certification (AWS Certified Solutions Architect, Azure Solutions Architect Expert, Google Cloud Certified Professional Cloud Architect)', 'System Design and Architecture for Cloud Environments']",[],4-6 months


In [23]:
import requests

def fetch_github_projects(query, limit=8):
    url = f"https://api.github.com/search/repositories?q={query}+project+in:name,description&sort=stars&order=desc"
    headers = {"Accept": "application/vnd.github+json"}
    r = requests.get(url, headers=headers)
    data = r.json()
    repos = []
    for repo in data.get("items", [])[:limit]:
        repos.append({
            "title": repo["name"],
            "description": repo.get("description", "No description."),
            "stars": repo["stargazers_count"],
            "url": repo["html_url"]
        })
    return repos

In [24]:
def fetch_public_apis(limit=5):
    backup_url = "https://raw.githubusercontent.com/davemachado/public-api/master/data.json"
    try:
        r = requests.get(backup_url)
        r.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        data = r.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching public APIs: {e}")
        return []
    except json.JSONDecodeError:
        print("Error decoding JSON from public API list. The data might not be in the expected format.")
        print("Response content:", r.text[:500]) # Print first 500 characters for debugging
        return []

    apis = []
    for cat, entries in data.items():
        for e in entries[:limit]:
            apis.append({
                "api": e.get("API"),
                "description": e.get("Description"),
                "link": e.get("Link"),
                "category": cat
            })
    return apis

In [25]:
model = genai.GenerativeModel("gemini-2.0-flash")

user_skills = input("🧠 Your Skills: ")
career_goal = input("🎯 Career Goal: ")

github_projects = fetch_github_projects(career_goal, 6)
public_apis = fetch_public_apis(5)

prompt_projects = f"""
You are an AI mentor recommending real project ideas.

User Info:
Skills: {user_skills}
Career Goal: {career_goal}

Here are some trending GitHub projects:
{github_projects}

Here are some available Public APIs:
{public_apis}

Suggest 5 personalized project ideas (some based on GitHub repos, some new using these APIs).
Return JSON:
[
  {{
    "title": "Project Title",
    "description": "Short summary",
    "difficulty": "Beginner/Intermediate/Advanced",
    "why_recommended": "Why it's ideal for user",
    "link": "GitHub or API URL"
  }}
]
"""

response_proj = model.generate_content(prompt_projects)
print("✅ Gemini Project Recommendations:\n")
print(response_proj.text)


🧠 Your Skills: python, nextjs, mongodb
🎯 Career Goal: ml engineer
Error fetching public APIs: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/davemachado/public-api/master/data.json
✅ Gemini Project Recommendations:

```json
[
  {
    "title": "Heart Failure Prediction Web App with Enhanced UI",
    "description": "Extend the 'Heart-Failure-Prediction' project by improving the UI using Next.js, adding features like user authentication, historical data visualization, and A/B testing of different ML models.",
    "difficulty": "Intermediate",
    "why_recommended": "Builds on an existing ML project while incorporating Next.js for front-end development and expands your ML skillset with model comparison.  Connects directly to your ML engineer goal.",
    "link": "https://github.com/DavieObi/Heart-Failure-Prediction"
  },
  {
    "title": "Document Intelligence Web App with MongoDB Storage and User Roles",
    "description": "Fork the 'doc-intelligence-in-a-box' proje

In [26]:
import json, pandas as pd

json_string = response_proj.text.strip()

# Find the start and end indices of the JSON block
json_start = json_string.find("```json")
json_end = json_string.rfind("```")

if json_start != -1 and json_end != -1:
    # Extract the JSON string between the markers
    json_string = json_string[json_start + 7:json_end].strip()
else:
    # If markers are not found, try to parse the whole string (might be pure JSON)
    pass # Or handle this case as an error if expected to always have markers


project_data = json.loads(json_string)
df_proj = pd.DataFrame(project_data)
df_proj.style.background_gradient(subset=[], cmap="Blues")

,title,description,difficulty,why_recommended,link
0,Heart Failure Prediction Web App with Enhanced UI,"Extend the 'Heart-Failure-Prediction' project by improving the UI using Next.js, adding features like user authentication, historical data visualization, and A/B testing of different ML models.",Intermediate,Builds on an existing ML project while incorporating Next.js for front-end development and expands your ML skillset with model comparison. Connects directly to your ML engineer goal.,https://github.com/DavieObi/Heart-Failure-Prediction
1,Document Intelligence Web App with MongoDB Storage and User Roles,"Fork the 'doc-intelligence-in-a-box' project and enhance it with user roles (admin, user), a more intuitive Next.js UI, and replace Azure Cosmos DB with MongoDB for document storage. Implement a search/filtering feature within the UI.",Intermediate,"This project allows you to apply your Next.js and MongoDB skills within a document processing ML context, while contributing to an existing project. It addresses data persistence and security concerns.",https://github.com/Azure-Samples/doc-intelligence-in-a-box
2,Personalized Learning Path Tracker based on 'Becoming-ML_engineer',"Create a web app (Next.js frontend, MongoDB backend) that allows users to track their progress towards becoming an ML engineer, similar to 'Becoming-ML_engineer'. Users can add projects, books, courses, and track their completion. Implement a recommendation system that suggests resources based on user's skills and interests.",Intermediate,This project allows you to practice front-end and back-end development while focusing on a topic relevant to your career goals. It also introduces you to the basics of recommendation systems.,https://github.com/manvimadandotai/Becoming-ML_engineer
3,ML Model Deployment Pipeline with Monitoring Dashboard,"Develop a simple ML model (e.g., sentiment analysis) and create a deployment pipeline using Docker, a cloud service like AWS or GCP, and a monitoring dashboard with Next.js. The dashboard should show model performance metrics (accuracy, latency), resource usage, and any errors. Use MongoDB to store model versions and deployment logs.",Advanced,This project addresses a critical aspect of being an ML engineer: deploying and monitoring models in production. It integrates all your skills and introduces you to DevOps concepts.,None
4,Contribute to 'ml-class' by building a new interactive lesson,"Contribute to the 'ml-class' repository by creating a new interactive lesson or project. This could involve implementing a new ML algorithm, creating a data visualization tool, or building a simple web app that demonstrates a specific ML concept. Focus on using clear and concise explanations and engaging visuals.",Beginner/Intermediate,"This project is a great way to give back to the community, learn new ML concepts, and improve your communication skills. It's particularly beneficial because you can contribute to a well-structured educational resource.",https://github.com/lukas/ml-class


In [27]:
df_proj.to_pickle("project_recommendations.pkl")
print("✅ Project recommendations saved as project_recommendations.pkl")

✅ Project recommendations saved as project_recommendations.pkl
